# Калибровка классификаторов

Проблема — классификатор выдает какие-то числа от 0 до 1 (или просто вещественные числа).
Эти числа, на самом деле, *уверенность* классификатора, а мы бы хотели иметь *вероятности*.

Для простоты давайте ограничемся задачей бинарнокй классификации


## Как определить вероятность?

Пусть $a(X)$ — предсказание классификатора.
Мы бы хотели, чтобы выполнялось равенство $P(y | a(X) = \hat{p}) = \hat{p}$.
На практике, однако, часто оказывается, что есть ровно один объект, для котрого выполняется $a(X) = \hat{p}$, и поэтому говорить о вероятностях не имеет смысла.

### Разбиение на группы
Можно разбить отрезок $[0; 1]$ на подотрезки и посчитать в каждом из них среднее значение классифиатора и сравнить его с долей объектов класса 1.

Кривую, построенную в таких координатах, называют *диаграммой калибровки*.

У идеально откалиброванного классификатора диаграма представляет собой прямую $y = x$.

(диаграмы взяты из Учебника Яндекса) \[1\]

![Пример диаграммы калибровки](https://yastatic.net/s3/ml-handbook/admin/12_2_32a327a143.png)


#### Пример слишком уверенного классификатора
![overconfident](https://yastatic.net/s3/ml-handbook/admin/12_3_033f331fed.png)


#### Пример неуверенного классификатора
![underconfident](https://yastatic.net/s3/ml-handbook/admin/12_4_d557d0984e.png)

## Оценка качества калибровки

### Expected calibration error 
Разобьем отрезок $[0; 1]$ на подотрезки $B_1, B_2, \ldots, B_k$ и вычислим
$$
    \sum\limits_{j=1}^k \frac{\#B_j}{N} |\bar{y}(B_i) - \bar{a}(B_i)|,
$$ 
где $\#B_j$ - количетсво элементов с предсказаниями в отрезке $B_j$, $\bar{y}(B_i)$ — их доля, приндалжещаих первому классу, $\bar{a}(B_i)$ — среднее предсказание классификатора на них.


### Max calibration error
В обозначениях из предыдущего пункта вычислим
$$
    \max\limits_{j=1, 2,\ldots, k} |\bar{y}(B_i) - \bar{a}(B_i)|.
$$ 

Так же можно взять среднеквадратичное.


### Brier score
Вычислим 
$$
    \sum\limits_{j=1}^N (y_j - a(x_j))^2
$$
или
$$
    \sum\limits_{j=1}^N \left(y_j\log a(x_j) + (1 - y_j)(\log(1 - a(x_j))\right)
$$


## Алгоритмы калибровки

Функцию, которая из исходных преобразований, получает откалиброванные, назовем *функцией деформации*.

### Гистограмная калибровка
Разобьем отрезок на подотрезки $B_i, i=1, 2,\ldots, k$, и если предсказание классификатора попало в $i$-отрезок, то будем предсказывать значение $\theta_i$.
Для подбора параметров $\theta_i$ решается задача оптимизации
$$
    \sum\limits_{j=1}^k \sum\limits_{i: a(i)\in B_j} (\theta_j - y_i))^2 \to\min
$$.
Замечания:
- надо как-то выбирать бины и их количество;
- можно брать сумму модулей, а не сумму квадратов
- можно выбирать бины равной длины, или разной (и нормировать каждое слагаемое на длину), или равномощным образом (и нормировать на число элементов)
- конечное число выходов у итогового классификатора

### Изотоническая регрессия
Решаем похожую задачу, но теперь 
1) настраиваем границы бинов $b_1, b_2, \ldots b_{k-2}$.
2) накладываем ограничение $\theta_1 < \theta_2 < \ldots < \theta_k$.

Функций деформации $g$ по-прежнему кусочно-постоянная.

Решается оптимизационная задача
$$
    \sum\limits_{i=1}^N (y_i - g(a(x_i))^2 \to\min
$$

Может быть решена за линейное время с помощью алгоритма Pool Adjacent Violators Algorithm

### Калибровка Платта
Метод предложен в [4], изначально разрабатывался для SVM, но может быть применен с к другим алгоритмам.

У нас есть способ, как превратить произвольное вещественное число в число из отрезка $[0; 1]$ — логистическая функция.
Положим
    $$P(y = 1 | a(x) = p) = \frac{1}{1 + e^{-A \cdot p -b}}.$$
    
Параметры $A$ и $B$ можно подобрать, оптимизирую кросс-энтропию на отложенной выборке (или кросс-валидацией).
   
   
   
### Beta-калибровка
Функция деформация ищется в виде
$$
    \mu(s; a, b, c) = \cfrac{1}{1 + \cfrac{1}{e^c\frac{s^a}{(1 - s)^b}}},
$$
где $a, b, c$ — настраиваемые параметры.

Авторы \[5\] показывают, что их можно получить, решив задачу логистичесокй регресии на признаках 
$\ln(q_i)$ и $-\ln(1 - q_i)$.

## Многоклассовый случай
### Softmax с температурой
$$
    a(x) = Softmax(\frac{b_1}{T}, \frac{b_2}{T}, \ldots, \frac{b_l}{T})
$$

### Калибровка с помощью распределение Дирехле \[6\]
Распределение Дирехле
$$
    f(x_1, x_2, \ldots, x_K; \alpha_1, \alpha_2, \ldots, \alpha_K) = 
    \frac{1}{B(\alpha_1, \alpha_2, \ldots, \alpha_K)} \prod\limits_{i=1}^K x_i^{\alpha_i - 1},
$$
где $B(\alpha_1, \alpha_2, \ldots, \alpha_K) = 
\cfrac{\prod\limits_{i=1}^K \Gamma(\alpha_i)}{\Gamma\left(\sum\limits_{j=1}^k \alpha_j\right)}$


[Визуализация](https://chart-studio.plotly.com/~david_avakian/14.embed)


Функция деформации может быть представлена в виде 
$$
    \mu(\mathbf{q}; \mathbf{W}, \mathbf{b}) = Softmax(\mathbf{W}\ln \mathbf{q} + \mathbf{b})
$$


## Список литературы
1. [Статья в учебнике Яндекса](https://academy.yandex.ru/handbook/ml/article/kak-ocenivat-veroyatnosti)
2. [Статья от Александра Дьяконова](https://alexanderdyakonov.wordpress.com/2020/03/27/%D0%BF%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D0%B0-%D0%BA%D0%B0%D0%BB%D0%B8%D0%B1%D1%80%D0%BE%D0%B2%D0%BA%D0%B8-%D1%83%D0%B2%D0%B5%D1%80%D0%B5%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D0%B8/)
3. [Tutorial на ECML KDD 2020](https://classifier-calibration.github.io/)
4. Platt, J. (1999). Probabilistic outputs for support vector machines and comparison to regularized likelihood methods. Advances in Large Margin Classifiers (pp. 61–74).
5. Kull, M., Silva Filho, T. M., Flach, P., et al.  Beyond sigmoids: How to obtain well-calibrated probabilities frombinary classifiers with beta calibration. Electronic Journalof Statistics, 11(2):5052–5080, 2017 [Link](https://research-information.bris.ac.uk/ws/portalfiles/portal/154625753/Full_text_PDF_final_published_version_.pdf)
6. Kull, M. et. al. Beyond temperature scaling: Obtaining well-calibrated multi-class probabilities with Dirichlet calibration. InProceedings of Advances in Neural Information Processing Systems, pp. 12295–12305, 2019. [Link](https://arxiv.org/abs/1910.12656)